# 2. 一般化線形回帰（問題21～33）

In [1]:
import copy
import japanize_matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy
from matplotlib.pyplot import imshow
from numpy.random import randn
from scipy import stats

In [5]:
# Linuxでの実行（Windows, Google Colaboratory不可）
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glmnet_python
from glmnet import glmnet
from glmnetCoef import glmnetCoef
import sys
from cvglmnet import cvglmnet
from cvglmnetCoef import cvglmnetCoef
from cvglmnetPlot import cvglmnetPlot
import os
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr
from lifelines import KaplanMeierFitter

ModuleNotFoundError: No module named 'glmnet'

### 21

In [ ]:
def f(x):
    return (np.exp(beta_0 + np.dot(beta, x))
            / (1 + np.exp(beta_0 + np.dot(beta, x))))

In [ ]:
beta_0 = 0
beta_seq = np.array([0, 0.2, 0.5, 1, 2, 10])
m = len(beta_seq)
x = np.arange(-10, 10)
for i in range(m):
    beta = beta_seq[i]
    plt.plot(x, f(x), label=beta_seq[i])
plt.title("ロジスティック曲線")
plt.xlabel("$x$")
plt.ylabel("$P(Y=1|x)$")
plt.legend()

### 22

In [ ]:
# データ生成
N = 100
p = 2
X = np.random.randn(N, p)
X = np.concatenate([np.ones(N).reshape(N, 1), X], axis=1)
beta = np.random.randn(p+1)
y = np.zeros(N)
s = np.dot(X, beta)
prob = 1 / (1 + np.exp(s))
for i in range(N):
    if np.random.rand(1) > prob[i]:
        y[i] = 1
    else:
        y[i] = -1
beta

In [ ]:
# 最尤推定値の計算
beta = np.inf
gamma = np.random.randn(p + 1)
while np.sum((beta - gamma) ** 2) > 0.001:
    beta = gamma.copy()
    s = np.dot(X, beta)
    v = np.exp(-s * y)
    u = y * v / (1 + v)
    w = v / (1 + v) ** 2
    z = s + u / w
    W = np.diag(w)
    gamma = np.dot(np.linalg.inv(X.T @ W @ X), np.dot(X.T @ W, z))
    print(gamma)
beta  # 真の値。最尤法でこの値を推定したい

### 24

In [ ]:
# Linuxマシンのフォルダに"breastcancer.csv"をおく
x = np.loadtxt("breastcancer.csv", delimiter=",", skiprows=1,
               usecols=range(1000))
y = np.loadtxt("breastcancer.csv", delimiter=",", skiprows=1,
               dtype="unicode", usecols=1000)
n = len(y)
yy = np.ones(n)
for i in range(n):
    if y[i] == "control":
        yy[i] = 1
    else:
        yy[i] = -1
fit1 = cvglmnet(x=x.copy(), y=yy.copy(), ptype="deviance", family="binomial")

### 26

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X = np.array(iris["data"])
y = np.array(iris["target"], dtype='float64')
cvfit3 = cvglmnet(x=X.copy(), y=y.copy(),
                  ptype='deviance', family='multinomial')
lam_min = (cvfit1['lambda_min'])
beta = cvglmnetCoef(cvfit)
print(lam_min)
print(beta)

In [ ]:
fig3 = plt.figure()
cvglmnetPlot(cvfit3)
fig3.savefig("img3.png")

In [ ]:
K = 3
p = 5
n = 150
gamma = np.zeros((K, p))
for k in range(K):
    for j in range(p):
        gamma[k, j] = np.sum(beta[k][j])
v = np.zeros(n)
for i in range(n):
    max_value = -np.inf
    for k in range(K):
        value = ## 空欄 ##
        if value > max_value:
            v[i] = k
            max_value = value
table_count(3, y, v)

### 28

In [ ]:
def poisson_lasso(X, y, lam):
    p = X.shape[1]   # pはすべて1の列を含んでいる
    beta = np.random.randn(p)
    gamma = np.random.randn(p)
    while np.sum((beta - gamma) ** 2) > 0.0001:
        beta = gamma
        s = np.dot(X, beta)
        w = ## 空欄(1) ##
        u = ## 空欄(2) ##
        z = ## 空欄(3) ##
        gamma_0, gamma_1 = W_linear_lasso(
            X[:, range(1, p)], z, np.diag(w), lam)
        gamma = np.block([gamma_0, gamma_1]).copy()
        print(gamma)
    return gamma

In [ ]:
N = 100    # lambdaの値が小さいと発散して，推定値が出ないことがある。
p = 3
X = np.random.randn(N, p)
X = np.concatenate([np.ones(N).reshape(N, 1), X], axis=1)
beta = np.random.randn(p + 1)
s = np.dot(X, beta)
y = np.random.poisson(lam=np.exp(s))
print(beta)

In [ ]:
poisson_lasso(X, y, 2)

### 29

In [ ]:
import pandas as pd
from lifelines import KaplanMeierFitter

In [ ]:
df = pd.read_csv("kidney.csv")
df.drop(df.columns[0], axis=1, inplace=True)
df

In [ ]:
kmf = KaplanMeierFitter()
ax = None
for name, group in df.groupby("disease"):
    kmf.fit(group["time"], event_observed=group["status"],
            label="肝臓病の種類：" + str(name))
    if ax is None:
        ax = kmf.plot()
    else:
        ax = kmf.plot(ax=ax)
plt.title("Kaplan-Meier Curve")
plt.show()

### 32(a)

In [ ]:
# Linuxでの実行（Windows, Google Colaboratory不可）
# 最適な係数を求める（7339個中29個のみが非ゼロ）
base = importr("base")
base.load("LymphomaData.rda")
w = robjects.globalenv["patient.data"]
X = np.array(w[0]).T
y = np.array(w[1])
delta = np.array(w[2])
v = ## 空欄 ##
fit = glmnet(x=X.copy(), y=v.copy(), family="cox")
beta = glmnetCoef(fit, s=np.float64([0.119787]))
print(np.sum(beta != 0))

### 32(b)

In [ ]:
# Linuxでの実行（Windows, Google Colaboratory不可）
# Kaplan-Meier曲線を描く
z = np.sign(np.dot(X, beta))
df2 = pd.DataFrame(np.concatenate([y.reshape(240, 1),
                                   delta.reshape(240, 1), z], axis=1))
df2.columns = ["time", "status", "sign"]
fig = plt.figure()
kmf = KaplanMeierFitter()
ax = None
for name, group in df2.groupby("## 空欄(1) ##"):
    kmf.fit(group["## 空欄(2) ##"], event_observed=group["## 空欄(3) ##"],
            label="z = " + str(name))
    if ax is None:
        ax = kmf.plot()
    else:
        ax = kmf.plot(ax=ax)

In [ ]:
plt.title("Kaplan-Meier Curve")
fig.savefig("img7.png")

### 33